# Classificador Regressão Logística - Experimento

Este componente treina um modelo de Regressão Logística para classificação usando [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). <br>
Scikit-learn é uma biblioteca open source de machine learning que suporta apredizado supervisionado e não supervisionado. Também provê várias ferramentas para ajustes de modelos, pré-processamento de dados, seleção e avaliação de modelos, além de outras funcionalidades.

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [ ]:
# parameters
dataset = "/tmp/data/iris.csv" #@param {type:"string"}
target = "Species" #@param {type:"feature", label:"Atributo alvo", description: "Seu modelo será treinado para prever os valores do alvo."}

# selected features to perform the model
filter_type = "remover" #@param ["incluir","remover"] {type:"string",multiple:false,label:"Modo de seleção das features",description:"Se deseja informar quais features deseja incluir no modelo, selecione a opção 'incluir'. Caso deseje informar as features que não devem ser utilizadas, selecione 'remover'. "}
model_features = "" #@param {type:"feature",multiple:true,label:"Features para incluir/remover no modelo",description:"Seu modelo será feito considerando apenas as features selecionadas. Caso nada seja especificado, todas as features serão utilizadas"}

# features to apply Ordinal Encoder
ordinal_features = "" #@param {type:"feature",multiple:true,label:"Features para fazer codificação ordinal", description: "Seu modelo utilizará a codificação ordinal para as features selecionadas. As demais features categóricas serão codificadas utilizando One-Hot-Encoding."}

# hyperparameters
penalty = "l2" #@param ["l1, "l2", "elasticnet", "None"] {type:"string", label:"Penalidade", description:"Norma utilizada na penalização do erro"}
C = 1.0 #@param {type:"number", label:"Regularização Inversa", description:"Retém a modificação de força da regularização ao ser posicionada inversamente no regulador Lambda"}
fit_intercept = True #@param {type"boolean", label:"Interceptação", description:"Especifica se uma constante (viés ou interceptação) deve ser adicionada à função de decisão"}
class_weight = None #@param ["balanced”, “balanced_subsample”] {type:"string", label:"Peso das Classes", description:"Especifica pesos de amostras quando for ajustar classificadores como uma função da classe do target"}
solver = "liblinear" #@param ["liblinear", "lbfgs", "sgd", "adam"] {type:"string", label:"Solucionador", description:"Algoritmo a ser usado no problema de otimização"}
max_iter = 100 #@param {type: "integer", label:"Iterações", description:"Número máximo de itereações feitas para os solvers convergirem"}
multi_class = "auto" #@param ["auto", "ovr", "multimomial"] {type:"string", label:"Multiclasse", description:"Classificação com mais de duas classes, porém cada amostra pode ser rotulada apenas como uma classe"}

# predict method
method = "predict_proba" #@param ["predict_proba", "predict"] {type:"string", label:"Método de Predição", description:"Se optar por 'predict_proba', o método de predição será a probabilidade estimada de cada classe, já o 'predict' prediz a qual classe pertence"} 

# Plots to ignore
plots_ignore = [""] #@param ["Dados de Teste", "Matriz de Confusão", "Métricas Comuns", "Curva ROC", "Tabelas de Dados", "SHAP"] {type:"string",multiple:true,label:"Gráficos a serem ignorados", description: "Diversos gráficos são gerados ao executar o treinamento e validação do modelo, selecione quais não devem ser gerados."}

## Acesso ao conjunto de dados

O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.<br>
O tipo da variável retornada depende do arquivo de origem:
- [pandas.DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
- [Binary IO stream](https://docs.python.org/3/library/io.html#binary-i-o) para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [ ]:
import pandas as pd

df = pd.read_csv(dataset)

## Acesso aos metadados do conjunto de dados

Utiliza a função `stat_dataset` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para carregar metadados. <br>
Por exemplo, arquivos CSV possuem `metadata['featuretypes']` para cada coluna no conjunto de dados (ex: categorical, numerical, or datetime).

In [ ]:
import numpy as np
from platiagro import stat_dataset

metadata = stat_dataset(name=dataset)
featuretypes = metadata["featuretypes"]

columns = df.columns.to_numpy()
featuretypes = np.array(featuretypes)
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)
featuretypes = np.delete(featuretypes, target_index)

## Remoção de linhas com valores faltantes no atributo alvo

Caso haja linhas em que o atributo alvo contenha valores faltantes, é feita a remoção dos casos faltantes.

In [ ]:
df.dropna(subset=[target], inplace=True)
y = df[target].to_numpy()

## Filtragem das features 

Seleciona apenas as features que foram declaradas no parâmetro model_features. Se nenhuma feature for especificada, todo o conjunto de dados será utilizado para a modelagem.

In [ ]:
columns_to_filter = columns

if len(model_features) >= 1:

    if filter_type == "incluir":
        columns_index = (np.where(np.isin(columns, model_features)))[0]
        columns_index.sort()
        columns_to_filter = columns[columns_index]
        featuretypes = featuretypes[columns_index]
    else:
        columns_index = (np.where(np.isin(columns, model_features)))[0]
        columns_index.sort()
        columns_to_filter = np.delete(columns, columns_index)
        featuretypes = np.delete(featuretypes, columns_index)

# keep the features selected
df_model = df[columns_to_filter]
X = df_model.to_numpy()

## Codifica labels do atributo alvo

As labels do atributo alvo são convertidos em números inteiros ordinais com valor entre 0 e n_classes-1. 

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

## Divide dataset em subconjuntos de treino e teste

Subconjunto de treino: amostra de dados usada para treinar o modelo.<br>
Subconjunto de teste: amostra de dados usada para fornecer uma avaliação imparcial do treinamento do modelo no subconjunto de dados de treino.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

## Configuração das features

In [ ]:
from platiagro.featuretypes import NUMERICAL

# Selects the indexes of numerical and non-numerical features
numerical_indexes = np.where(featuretypes == NUMERICAL)[0]
non_numerical_indexes = np.where(~(featuretypes == NUMERICAL))[0]

# Selects non-numerical features to apply ordinal encoder or one-hot encoder
ordinal_features = np.array(ordinal_features)

non_numerical_indexes_ordinal = np.where(
    ~(featuretypes == NUMERICAL) & np.isin(columns_to_filter, ordinal_features)
)[0]

non_numerical_indexes_one_hot = np.where(
    ~(featuretypes == NUMERICAL) & ~(np.isin(columns_to_filter, ordinal_features))
)[0]

# After the step handle_missing_values,
# numerical features are grouped in the beggining of the array
numerical_indexes_after_handle_missing_values = np.arange(len(numerical_indexes))

non_numerical_indexes_after_handle_missing_values = np.arange(
    len(numerical_indexes), len(featuretypes)
)

ordinal_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[
    np.where(np.isin(non_numerical_indexes, non_numerical_indexes_ordinal))[0]
]

one_hot_indexes_after_handle_missing_values = non_numerical_indexes_after_handle_missing_values[
    np.where(np.isin(non_numerical_indexes, non_numerical_indexes_one_hot))[0]
]

## Treina modelo usando sklearn.linear_model.LogisticRegression

In [ ]:
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipeline = Pipeline(
    steps=[
        (
            "handle_missing_values",
            ColumnTransformer(
                [
                    ("imputer_mean", SimpleImputer(strategy="mean"), numerical_indexes),
                    (
                        "imputer_mode",
                        SimpleImputer(strategy="most_frequent"),
                        non_numerical_indexes,
                    ),
                ],
                remainder="drop",
            ),
        ),
        (
            "handle_categorical_features",
            ColumnTransformer(
                [
                    (
                        "feature_encoder_ordinal",
                        OrdinalEncoder(),
                        ordinal_indexes_after_handle_missing_values,
                    ),
                    (
                        "feature_encoder_onehot",
                        OneHotEncoder(),
                        one_hot_indexes_after_handle_missing_values,
                    ),
                ],
                remainder="passthrough",
            ),
        ),
        (
            "estimator",
            LogisticRegression(
                solver=solver,
                penalty=penalty,
                C=C,
                fit_intercept=fit_intercept,
                class_weight=class_weight,
                max_iter=max_iter,
                multi_class=multi_class,
            ),
        ),
    ]
)

pipeline.fit(X_train, y_train)

features_after_pipeline = np.concatenate(
    (columns_to_filter[numerical_indexes], columns_to_filter[non_numerical_indexes])
)

## Visualização de desempenho

A [**Matriz de Confusão**](https://en.wikipedia.org/wiki/Confusion_matrix) (Confusion Matrix) é uma tabela que nos permite a visualização do desempenho de um algoritmo de classificação. <br>
É extremamente útil para mensurar [Accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision#In_binary_classification), [Recall, Precision, and F-measure](https://en.wikipedia.org/wiki/Precision_and_recall).

In [ ]:
from sklearn.metrics import confusion_matrix

# uses the model to make predictions on the Test Dataset
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)

# computes confusion matrix
labels = np.unique(y)
data = confusion_matrix(y_test, y_pred, labels=labels)

# puts matrix in pandas.DataFrame for better format
labels_dec = label_encoder.inverse_transform(labels)
confusion_matrix = pd.DataFrame(data, columns=labels_dec, index=labels_dec)

## Salva métricas

Utiliza a função `save_metrics` do [SDK da PlatIAgro](https://platiagro.github.io/sdk/) para salvar métricas. Por exemplo: `accuracy`, `precision`, `r2_score`, `custom_score` etc.<br>

In [ ]:
from platiagro import save_metrics

save_metrics(confusion_matrix=confusion_matrix)

## Visualiza resultados
A avaliação do desempenho do modelo pode ser feita por meio da análise da Curva ROC (ROC). Esse gráfico permite avaliar a performance de um classificador binário para diferentes pontos de cortes. A métrica AUC (Area under curve) também é calculada e indicada na legenda do gráfico. Se a variável resposta tiver mais de duas categorias, o cálculo da curva ROC e AUC é feito utilizando o algoritmo one-vs-rest, ou seja, calcula-se a curva ROC e AUC de cada classe em relação ao restante.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from platiagro.plotting import plot_classification_data

if "Dados de Teste" not in plots_ignore:
    
    plot_classification_data(pipeline, columns_to_filter, X_train, y_train, X_test, y_test, y_pred)
    plt.show()

In [ ]:
from platiagro.plotting import plot_matrix

if "Matriz de Confusão" not in plots_ignore:
    
    plot_matrix(confusion_matrix)
    plt.show()

In [ ]:
from platiagro.plotting import plot_common_metrics

if "Métricas Comuns" not in plots_ignore:
    
    plot_common_metrics(y_test, y_pred, labels, labels_dec)
    plt.show()

In [ ]:
from platiagro.plotting import plot_roc_curve

if "Curva ROC" not in plots_ignore:
    
    plot_roc_curve(y_test, y_prob, labels_dec)
    plt.show()

Por meio da bibiloteca SHAP retornamos o gráfico abaixo, através do qual é possível visualziar o impacto das features de entrada para cada possível classe na saída.

In [ ]:
from platiagro.plotting import plot_shap_classification_summary

if "SHAP" not in plots_ignore:

    plot_shap_classification_summary(pipeline, X_train,y_train,columns,len(columns),label_encoder)

Visualização da tabela contendo os resultados finais

In [ ]:
from platiagro.plotting import plot_data_table

if "Tabelas de Dados" not in plots_ignore:

    df_test = pd.DataFrame(X_test, columns=columns_to_filter)
    labels_prob = list(label_encoder.inverse_transform(np.unique(y)))
    labels_prob = [str(label) +'_prob' for label in labels_prob]
    new_cols = [target] + labels_prob

    y_pred_expanded = np.expand_dims(label_encoder.inverse_transform(y_pred), axis=1)
    y_arrays = np.concatenate((y_pred_expanded, y_prob), axis=1)
    for col, y_array  in zip(new_cols,y_arrays.T):
        df_test[col] = y_array

    ax = plot_data_table(df_test)
    plt.show()

## Salva alterações no conjunto de dados

O conjunto de dados será salvo (e sobrescrito com as respectivas mudanças) localmente, no container da experimentação, utilizando a função `pandas.DataFrame.to_csv`.<br>

In [ ]:
from re import sub

pipeline.fit(X, y)

y_prob = pipeline.predict_proba(X)
y_pred = pipeline.predict(X)

new_columns = []
new_columns.append("Logistic_predict_class")
y_pred = label_encoder.inverse_transform(y_pred)
df[new_columns[-1]] = y_pred

for i, class_j in zip(range(len(labels_dec)), labels_dec):
    new_columns.append(
        sub("[^a-zA-Z0-9\n\.]", "_", str("Logistic_predict_proba" + "_" + str(class_j)))
    )
    df[new_columns[i]] = y_prob[:, i]

# save dataset changes
df.to_csv(dataset, index=False)

## Salva resultados da tarefa 

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.

In [ ]:
from joblib import dump

artifacts = {
    "pipeline": pipeline,
    "label_encoder": label_encoder,
    "columns": columns,
    "columns_to_filter": columns_to_filter,
    "new_columns": new_columns,
    "method": method,
    "features_after_pipeline": features_after_pipeline,
}

dump(artifacts, "/tmp/data/logistic-regression.joblib")